In [1]:
import importlib
import src.chat_bot
importlib.reload(src.chat_bot)
from src.chat_bot import PopiaAct

In [42]:

logging_level = 15
#logging_level = 20
log_file = ''

path_to_manual_as_csv_file = "./inputs/popia_manual.csv"
path_to_definitions_as_parquet_file = "./inputs/popia_definitions.parquet"
path_to_index_as_parquet_file = "./inputs/popia_index.parquet"


popia = PopiaAct(path_to_manual_as_csv_file, path_to_definitions_as_parquet_file, path_to_index_as_parquet_file, log_file=log_file, logging_level=logging_level)


In [33]:
print(popia.get_regulation_detail("56."))

56. Designation and delegation of deputy information officers
Each public and private body must make provision, in the manner prescribed in section 17 of the Promotion of Access to Information Act, with the necessary changes, for the designation of—
        (a) such a number of persons, if any, as deputy information officers as is necessary to perform the duties and responsibilities as set out in section 55(1) of thisAct; and
        (b) any power or duty conferred or imposed on an information officer by this Act to a deputy information officer of that public or private body.
Prior authorisation


In [47]:
#model_to_use="gpt-3.5-turbo"
#model_to_use = "gpt-4-1106-preview"
model_to_use="gpt-4"
#model_to_use="gpt-3.5-turbo-16k"

In [48]:
#user_context = "What is personal information?"
#user_context = "How should I protect personal information?"
#user_context = "What are the risks associated with storing personal information?"
#user_context = "Does POPIA apply to all industries?"
#user_context = "What does a data protection framework need to address?"



user_context = "As a medium size company, what do I need to do to comply with POPIA?"
user_context = "Do I need to employ additional staff?"
user_context = "What is an information officer?"
user_context = "Does every company need an information officer?"
user_context = "what do I need to do to comply with the act?"


popia.reset_conversation_history()
popia.user_provides_input(user_context, threshold=0.15, model_to_use=model_to_use, temperature=0, max_tokens=300)
print(popia.messages[-1]["content"])


INFO:src.chat_bot:User Question:
what do I need to do to comply with the act?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:src.chat_bot:#################   Similarity Search       #################
DEV:src.chat_bot:--   No relevant definitions found
DEV:src.chat_bot:--   Relevant Sections
DEV:src.chat_bot:0.0948:                  56.:          manual: What do I need to do to comply with POPIA?
DEV:src.chat_bot:0.1362:                  96.:        question: What can be done if I believe compliance with an enforcement notice is no longer necessary? 
DEV:src.chat_bot:0.1380:                  75.:        question: What are the requirements for submitting a complaint?
DEV:src.chat_bot:0.1420:                   6.:        question: What are the exemptions to Act?
DEV:src.chat_bot:0.1462:              109.(4):        question: What happens if I want to contest an allegation of non-compliance?
DEV:src.chat_bot:Top result: 56. with a cosine distance o

To comply with the act, you need to designate a certain number of persons as deputy information officers as necessary to perform the duties and responsibilities set out in the act. If an enforcement notice has been served, you may apply in writing to the Regulator for the cancellation or variation of that notice if there has been a change of circumstances. Any complaints to the Regulator must be made in writing. [Sections 56, 96, 75]


In [7]:
popia.messages

[{'role': 'user',
  'content': "Question: what does the regulation state about travel?\n\nSections from the Manual\nB.4 Single discretionary allowance and other miscellaneous payments for private individuals\n    (B) Travel allowances\n        (i) General\n            (a) There is no limit on the amount of Rand that may be spent on travelling within the CMA.\n            (b) Residents (natural persons) who are under the age of 18 years may not be accorded single discretionary allowance as outlined in subsection (A)(i) above, but may be accorded a travel allowance not exceeding an amount of R200 000 per calendar year.\n            (c) In addition, up to R25 000 in cash, per person, may be taken when proceeding on visits outside the CMA, to meet travellers' immediate needs on return to South Africa.\n            (d) When according travel allowances to persons who do not conduct accounts in their books, Authorised Dealers must record such persons' names, nationalities, residential address

In [26]:
import pandas as pd 
tmp_df = pd.read_parquet(path_to_index_as_parquet_file)


In [41]:
tmp_df.to_parquet(path_to_index_as_parquet_file, engine='pyarrow')

In [37]:
import src.embeddings
importlib.reload(src.embeddings)
from src.embeddings import get_ada_embedding, num_tokens_from_string


# Example values for the new row
question = 'What do I need to do to comply with POPIA?'
embedding = get_ada_embedding(question)

new_row = {
    'section': '56.',
    'text': question,
    'source': 'manual',
    'Embedding': embedding
}

# Adding the new row to the dataframe
tmp_df.loc[len(tmp_df)] = new_row

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [40]:
#(tmp_df[tmp_df["source"]=="question"])
#tmp_df.loc[486, "source"] = "manual"
tmp_df

,section,text,source,Embedding
0,2.,What is the purpose of the Protection of Perso...,question,"[-0.00897529348731041, -0.004865928087383509, ..."
1,2.,What rights does the Act provide individuals ...,question,"[-0.0016322160372510552, 0.0023074529599398375..."
2,2.,What kind of regulatory body is established b...,question,"[0.004339318256825209, -0.012121826410293579, ..."
3,3.,What data or information does POPIA apply to?,question,"[0.0023713144473731518, 0.0032313510309904814,..."
4,3.,How does the POPIA interact with other laws r...,question,"[0.003651085076853633, 0.010420947335660458, 0..."
...,...,...,...,...
482,112.,Regulations,heading,"[0.0018349532037973404, -0.012941248714923859,..."
483,113.,Procedure for making regulations,heading,"[0.009133221581578255, 0.004106280393898487, -..."
484,114.,Transitional arrangements,heading,"[0.00048234881251119077, -0.011674447916448116..."
485,115.,Short title and commencement,heading,"[-0.004542532376945019, -0.0018438507104292512..."


In [19]:
tmp_df['section'] = tmp_df['section'].str.replace(r'\.0$', '.', regex=True)

In [29]:
tmp_df[tmp_df["section"].str.endswith(".")]

,section,text,source,Embedding
0,2.,What is the purpose of the Protection of Perso...,question,"[-0.00897529348731041, -0.004865928087383509, ..."
1,2.,What rights does the Act provide individuals ...,question,"[-0.0016322160372510552, 0.0023074529599398375..."
2,2.,What kind of regulatory body is established b...,question,"[0.004339318256825209, -0.012121826410293579, ..."
3,3.,What data or information does POPIA apply to?,question,"[0.0023713144473731518, 0.0032313510309904814,..."
4,3.,How does the POPIA interact with other laws r...,question,"[0.003651085076853633, 0.010420947335660458, 0..."
...,...,...,...,...
481,111.,Fees,heading,"[-0.0012755942298099399, 0.006350830662995577,..."
482,112.,Regulations,heading,"[0.0018349532037973404, -0.012941248714923859,..."
483,113.,Procedure for making regulations,heading,"[0.009133221581578255, 0.004106280393898487, -..."
484,114.,Transitional arrangements,heading,"[0.00048234881251119077, -0.011674447916448116..."


In [24]:
len(tmp_df)

486

In [8]:
import importlib
import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_popia_manual_index
import re

index = get_popia_manual_index()


In [5]:
s = " B.4(B)(ii)(a) of the Manual states that foreign currency may be made available within the single discretionary allowance limit of R1 million per calendar year to eligible parties. However, the specific limits for adults are not mentioned in the provided sections. Please refer to the relevant section(s) for more information on the limits for travel allowances for adults."
#s = "B.4(B)(ii)(a)"

print(index.extract_valid_reference(s))


None


In [22]:
patterns = [
        r'[A-Z]\.\d{0,2}',
        r'\([A-Z]\)',
        r'\(\b(i|ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx|xxi|xxii|xxiii|xxiv|xxv|xxvi|xxvii)\b\)',
        r'\([a-z]\)',
        r'\([a-z]{2}\)',
        r'\((?:[1-9]|[1-9][0-9])\)'
    ]
matches = [re.findall(pattern, s) for pattern in patterns]
matches

[['B.4'], ['(B)'], ['ii'], ['(a)', '(s)'], ['(ii)'], []]

In [26]:
match = re.search(r'\([a-z]\)', s)
match

<re.Match object; span=(11, 14), match='(a)'>

In [30]:
s[match.end()]

' '

In [87]:
popia.get_regulation_detail("B.5(B)(iii)")

'B.5 Personal transfers by foreign nationals and immigrants\n    (B) Immigrants\n        (iii) Payments by immigrants\n            (a) Applications by immigrants may be approved by Authorised Dealers for the provision of foreign currency to cover current and arrear premiums due on foreign currency life insurance policies or contributions to pension and medical aid funds, provided that:\n                (aa) the necessary declaration and undertaking were completed; and\n                (bb) documentary evidence is presented, verifying the amounts due and that the commitment was entered into before the applicants took up residence in South Africa.\n            (b) Immigrants may against the presentation to an Authorised Dealer of documentary evidence confirming the amount involved repay loans received in their previous country of domicile.\n            (c) Immigrants may settle foreign tax commitments from South Africa against the production of documentary evidence confirming the amount 

In [11]:
print(popia._create_system_message())

You are answering questions for an Authorised Dealer (AD) based only on the relevant sections from the 'Currency and Exchange Manual for Authorised Dealers' (Manual or CEMAD) that are provided. You have three options:
1) Answer the question. Preface an answer with the tag 'ANSWER:'. If possible, end the answer with the reference to the section or sections you used to answer the question.
2) Request additional documentation. If, in the body of the sections provided, there is a reference to another section of the Manual that is directly relevant and not already provided, respond with the word 'SECTION:' followed by the section reference.
3) State 'NONE:' and nothing else in all other cases

Note: In the manual sections are numbered like A.1(A) or C.(C)(iii)(c)(cc)(3). The first index uses the regex pattern r'[A-Z]\.\d(0, 2)'. Thereafter, each sub-index is surrounded by round brackets


In [23]:
import pandas as pd
df_text_all = pd.read_parquet(path_to_index_as_parquet_file, engine='pyarrow')
df_text_all

,section,text,Embedding,source
0,Legal context,Legal context,"[0.008710513822734356, -0.01103331707417965, 0...",heading
1,A.2(A),Authorised entities. Authorised Dealers,"[-0.009998462162911892, -0.009398686699569225,...",heading
2,A.2(B),Authorised entities. Authorised Dealers in for...,"[-0.009065092541277409, -0.012516901828348637,...",heading
3,A.3(A),Duties and responsibilities of Authorised Deal...,"[0.009497863240540028, -0.01100919209420681, -...",heading
4,A.3(B),Duties and responsibilities of Authorised Deal...,"[-0.010621572844684124, -0.014073583297431469,...",heading
...,...,...,...,...
1493,J.(G),What is the procedure for reporting outward pa...,"[-0.0016078203916549683, -0.000447303900728002...",question
1494,J.(G),Rules for reporting outward payments using the...,"[0.005922725424170494, -0.004037913400679827, ...",summary
1495,B.4(G)(i),How much money can an individual take offshore?,"[0.01233571209013462, -0.03362925723195076, 0....",manual
1496,B.4(B)(i),Is there any general guidance on travel?,"[0.0002241560723632574, 0.000385825231205672, ...",manual


In [ ]:
popia.get_regulation_detail('I.(1)(E)(ii)')

'No section could be found with the reference I.(E)(ii)'

In [ ]:
#user_context = "What is a CFC Account?"
#user_context = "I am submitting documentation to the financial surveillance department. What information do I need to include in my submission?" # (A.3 (B))
#user_context = "Can a bank provide foreign currency denominated facilities to a South African Corporate?" #  (A.3 (B))
#user_context = "For how long does the approval for a facility remain valid?" #  (A.3 (B))
#user_context = "Other than authorised transaction requests, are there requirements for authorised dealers to submit information to the financial surveillance department?" #  (A.3 (B))
#user_context = "I have been asked to act on an approval issued to another bank. Can I do that?" #  (A.3 (B))
#####user_context = "--- Same as above but use "Authorised Dealer" instead of "Bank"
#user_context = "If I am concerned that as client may in breach of regulations, what should I do?" #  (A.3 (B))
#user_context = "What constitutes 'documentary evidence' for a foreign currency transaction?" #  (A.3 (C))
#user_context = "Is Swaziland part of the CMA?" #  (A.3 (E))
#user_context = "A client wants to buy Namibian Dollars. What approvals do they need before they can do this?" #  (A.3 (E))
#user_context = "A person from Lesotho wants to buy dollars from me. Can I sell them the dollars?" #  (A.3 (E))
#user_context = "A person from Namibia wants to buy rand from me. Can I sell them the rand?" #  (A.3 (E))
#user_context = "A treasury management company wants to trade dollars with me. What do I need to check before I can do this?" #  (A.4 (B))
#user_context = "A client wants to import goods and pay in foreign currency. What documentation do I need before I can assist with the transaction" #  (B.1 (B))
#user_context = "Over and above the price of the goods, what additional costs can be covered in foreign currency?" #  (B.1 (C))
#user_context = "Can advance payment be made in a foreign currency to cover imported goods?" #  (B.1 (D)(i))

#user_context = "Can advance payment be made in a foreign currency to cover imported capital goods?" #  (B.1 (D)(ii))
#user_context = "When paying for imports in cash, what documentation is required before providing the foreign currency?" #  (B.1 (D)(iii))

#user_context = "Can imported goods be financed over more than 12 months?" #  (B.1 (D)(iv))
#user_context = "How much money can an individual invest internationally?" #  ?
#user_context = "How much money can an individual take offshore?" #  ?

#user_context = "A media company want to list offshore. Are there specific conditions for this?"
# user_context = "How many rands can a resident take offshore in cash?"
#user_context = "Can I grant a foreign company a Rand facility to pay for goods they wish to purchase from a South African exporter?"
#user_context = "Can I support a non-residential entity with short-term rand trade finance for their exports from South Africa?"
#user_context = "Hi"
#user_context = "Who can trade gold?"
#user_context = "are there specific exchange control rules for religious organizations?"
#user_context = "How much foreign currency can a charity take offshore each year?"

